### Simple GEN AI App using Langchain

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ['LANGCHAIN_API_KEY']=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [6]:
## Data Ingestion -- from the website we need to scrape the data
from langchain_community.document_loaders import WebBaseLoader


In [ ]:
loader=WebBaseLoader("https://docs.smith.langchain.com/evaluation")
loader

In [8]:
docs=loader.load()
docs

[Document(metadata={'source': 'https://docs.smith.langchain.com/evaluation', 'title': 'Evaluation Quick Start | 🦜️🛠️ LangSmith', 'description': "Evaluations are a quantitative way to measure performance of LLM applications, which is important because LLMs don't always behave predictably — small changes in prompts, models, or inputs can significantly impact results. Evaluations provide a structured way to identify failures, compare changes across different versions of your application, and build more reliable AI applications.", 'language': 'en'}, page_content='\n\n\n\n\nEvaluation Quick Start | 🦜️🛠️ LangSmith\n\n\n\n\n\n\n\n\nSkip to main contentOur Building Ambient Agents with LangGraph course is now available on LangChain Academy!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationQuick StartTutorialsEvaluate a chatbotEvaluate a RAG applicationRun backtests on a new version of an agentRunning SWE-bench with LangSmithEvaluate a complex agentTest a ReA

In [10]:
### Load data--> docs --> divide text into chunks -->text--> vectors -->vector embeddings -->vector store db

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)

documents=text_splitter.split_documents(docs)

documents


[Document(metadata={'source': 'https://docs.smith.langchain.com/evaluation', 'title': 'Evaluation Quick Start | 🦜️🛠️ LangSmith', 'description': "Evaluations are a quantitative way to measure performance of LLM applications, which is important because LLMs don't always behave predictably — small changes in prompts, models, or inputs can significantly impact results. Evaluations provide a structured way to identify failures, compare changes across different versions of your application, and build more reliable AI applications.", 'language': 'en'}, page_content='Evaluation Quick Start | 🦜️🛠️ LangSmith'),
 Document(metadata={'source': 'https://docs.smith.langchain.com/evaluation', 'title': 'Evaluation Quick Start | 🦜️🛠️ LangSmith', 'description': "Evaluations are a quantitative way to measure performance of LLM applications, which is important because LLMs don't always behave predictably — small changes in prompts, models, or inputs can significantly impact results. Evaluations provide a s

In [11]:
from langchain_openai import OpenAIEmbeddings
embeddings= OpenAIEmbeddings()

In [12]:
from langchain_community.vectorstores import FAISS
vectorstoredb=FAISS.from_documents(documents,embeddings)

In [13]:
vectorstoredb

In [14]:
query="Evaluations are made up of three components:"
result=vectorstoredb.similarity_search(query)
result[0].page_content

"Evaluations are a quantitative way to measure performance of LLM applications, which is important because LLMs don't always behave predictably — small changes in prompts, models, or inputs can significantly impact results. Evaluations provide a structured way to identify failures, compare changes across different versions of your application, and build more reliable AI applications.\nEvaluations are made up of three components:"

In [15]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4o")

In [16]:
## Retrieval chain, Document Chain

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """
    Answer the following question based only on the provided context:
    <context>
    {context}
    </context>
    """
)

document_chain=create_stuff_documents_chain(llm,prompt)
document_chain



RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n    Answer the following question based only on the provided context:\n    <context>\n    {context}\n    </context>\n    '), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001B108C61A90>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001B108EF4440>, root_client=<openai.OpenAI object at 0x000001B108F78050>, root_async_client=<openai.AsyncOpenAI object at 0x000001B108F78690>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), 

In [18]:
from langchain_core.documents import Document
document_chain.invoke(
    {
        "input":"Evaluations are made up of",
        "context":[Document(page_content="Evaluations are made up of three components:A dataset with test inputs and optionally expected outputs.")]
    }
)

'Based on the provided context, evaluations consist of a dataset that includes test inputs and may also include expected outputs.'

However, we want the documents to first come from the retriever we just set up. That way, we can use the retriever to dynamically select the most relevant documents and pass those in for a given question.

### Input ---> Retriever ---> vectorestoredb

retriever is just like a interface.


In [20]:
retriever=vectorstoredb.as_retriever()
from langchain.chains import create_retrieval_chain

retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [21]:
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001B0E4499E80>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n    Answer the following question based only on the provided context:\n    <context>\n    {context}\n    </context>\n    '), additional_kwargs={})]

In [22]:
# get the response from the llm

response=retrieval_chain.invoke({"input":"Evaluations are made up of three components:"})
response['answer']

'Evaluations are a quantitative method to measure the performance of Large Language Model (LLM) applications. They help to identify failures, compare changes across different versions, and build more reliable AI applications. Evaluations consist of various components, including running evaluations with multimodal content, filtering experiments in the UI, and defining different types of evaluators such as LLM-as-a-judge or summary evaluators. They also involve handling aspects such as model rate limits, repetitions, and returning both categorical and numerical metrics. The provided context references tools like the LangSmith Experiments UI and guides on running these evaluations locally or using prebuilt evaluators.'

In [26]:
response['context']

[Document(id='29e49274-2d56-440a-a092-322f0e09100a', metadata={'source': 'https://docs.smith.langchain.com/evaluation', 'title': 'Evaluation Quick Start | 🦜️🛠️ LangSmith', 'description': "Evaluations are a quantitative way to measure performance of LLM applications, which is important because LLMs don't always behave predictably — small changes in prompts, models, or inputs can significantly impact results. Evaluations provide a structured way to identify failures, compare changes across different versions of your application, and build more reliable AI applications.", 'language': 'en'}, page_content="Evaluations are a quantitative way to measure performance of LLM applications, which is important because LLMs don't always behave predictably — small changes in prompts, models, or inputs can significantly impact results. Evaluations provide a structured way to identify failures, compare changes across different versions of your application, and build more reliable AI applications.\nEval